# Machine Learning Assignment: Regression Task

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style = 'whitegrid', color_codes = True)

import matplotlib
matplotlib.rc('xtick', labelsize = 40)
matplotlib.rc('ytick', labelsize = 40)

from statsmodels.tsa.stattools import acf, pacf


#### จะพยายามแบ่งการทำ Model เป็น 3 model ใหญ่ๆคือ ARIMA, Regression และ Dynamic Regression

## Data Preparation & Data Exploration

เลือกใช้ data จากภายนอกเพราะว่าต้องการค่าที่เป็นเวลาที่จริงขึ้น และสามารถนำข้อมูลอื่นๆ มาใช้ในการ Analysis และใช้ในการสร้าง model ชนิดอื่นด้วยโดย data ที่ได้เป็น data ที่ได้มาจากการ query ใน google bigquery

In [8]:
df_res = pd.read_csv('UK_results-20200503-150942.csv') # from bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide
df_mob = pd.read_csv('UK_mobility_results-20200503-152437.csv') # from bigquery-public-data.covid19_google_mobility.mobility_report